# 使用Deepgram转录音频文件并使用Anthropic准备面试问题！

**将此笔记本复制到您自己的驱动器中，然后按照下面的说明操作！** 🥳🥳🥳

----------------------------

# 开始使用：
运行以下三个单元格将允许您转录任何您想要的音频。下面的注释指出了您可以操作的变量来根据需要修改输出。

在运行此笔记本之前，您需要准备几个要转录的音频URL。您可以使用任何您想要的音频文件。

另外，如果您还没有注册Deepgram，请查看此处的链接：https://dpgr.am/prerecorded-notebook-signup

# 步骤1：依赖项

运行此单元格下载所有必要的依赖项。

注意：您可以通过单击左侧的播放按钮或单击单元格并同时按 `shift`+`ENTER` 来运行单元格。（或在Mac上按 `shift` + `return`）。

In [ ]:
! pip install requests ffmpeg-python
! pip install deepgram-sdk --upgrade
! pip install requests
! pip install anthropic

# 步骤2：音频URL文件

找到一些托管在服务器上的音频文件，以便您可以使用此笔记本。或者下面代码中提供了Deepgram提供的示例文件。

In [ ]:
# 您是否已完成上面的步骤2？ 👀
# 您在左侧的文件夹中看到您的音频文件了吗？ 📂

# 步骤3：转录

填写以下变量：


* `DG_KEY` = 您的个人Deepgram API密钥
* `AUDIO_FILE_URL` = 您希望转录的音频文件的URL。


现在运行单元格！（`Shift` + `Enter`）

-----------



另外，如果您已经是Deepgram用户，并且在此单元格中遇到错误，最常见的修复方法是：

1. 您可能需要更新您的deepgram-sdk安装。
2. 您可能需要检查您的Deepgram账户中还剩多少积分。

In [ ]:
from deepgram import DeepgramClient, PrerecordedOptions, FileSource
import requests

# Deepgram API密钥
DG_KEY = "🔑🔑🔑 您的API密钥在这里！ 🔑🔑🔑"

# 音频文件URL
AUDIO_FILE_URL = "https://static.deepgram.com/examples/nasa-spacewalk-interview.wav"

# 保存转录JSON文件的路径
TRANSCRIPT_FILE = "transcript.json"


def main():
    try:
        # 步骤1：使用API密钥创建Deepgram客户端
        deepgram = DeepgramClient(DG_KEY)

        # 从URL下载音频文件
        response = requests.get(AUDIO_FILE_URL)
        if response.status_code == 200:
            buffer_data = response.content
        else:
            print("下载音频文件失败")
            return

        payload: FileSource = {
            "buffer": buffer_data,
        }

        # 步骤2：配置Deepgram选项用于音频分析
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # 步骤3：使用文本有效负载和选项调用transcribe_file方法
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # 步骤4：将响应JSON写入文件
        with open(TRANSCRIPT_FILE, "w") as transcript_file:
            transcript_file.write(response.to_json(indent=4))

        print("转录JSON文件生成成功。")

    except Exception as e:
        print(f"异常：{e}")


if __name__ == "__main__":
    main()

如果上面的单元格成功，您应该在内容目录中看到JSON输出文件。注意：单元格运行完成和JSON文件实际出现之间可能会有短暂延迟。这是正常的。只需等待一会儿文件就会出现。

# 步骤4：查看您的转录

下面的函数解析输出JSON并打印出您刚刚转录的文件之一的转录！（确保
您尝试检查的文件确实已经加载到内容目录中。）

**将 `OUTPUT` 变量设置为您希望查看转录的文件的名称。**

然后运行此单元格（`Shift`+`Enter`）来查看音频的逐句转录！

In [ ]:
import json

# 将此变量设置为您希望读取的输出文件的路径
OUTPUT = "transcript.json"


# JSON加载了信息，但如果您只想读取转录文本，
# 请运行下面的代码！
def print_transcript(transcription_file):
    with open(transcription_file, "r") as file:
        data = json.load(file)
        result = data["results"]["channels"][0]["alternatives"][0]["transcript"]
        result = result.split(".")
        for sentence in result:
            print(sentence + ".")


print_transcript(OUTPUT)


如果上面的单元格成功，您应该看到音频转录的纯文本版本。

# 步骤5：使用Anthropic准备面试问题

现在我们可以将转录文本发送给Anthropic进行分析，帮助我们准备一些面试问题。运行下面的单元格（`Shift`+`Enter`）获取基于您上面音频转录的由Anthropic提供的一套建议面试问题。

In [ ]:
import anthropic
import json

transcription_file = "transcript.json"


# 从JSON文件获取转录文本的函数
def get_transcript(transcription_file):
    with open(transcription_file, "r") as file:
        data = json.load(file)
        result = data["results"]["channels"][0]["alternatives"][0]["transcript"]
        return result


# 从JSON文件加载转录文本
message_text = get_transcript(transcription_file)

# 初始化Claude API客户端
client = anthropic.Anthropic(
    # 默认为os.environ.get("ANTHROPIC_API_KEY")
    # Claude API密钥
    api_key="🔑🔑🔑 您的API密钥在这里！ 🔑🔑🔑"
)

# 为API请求准备文本
formatted_messages = [{"role": "user", "content": message_text}]

# 生成深思熟虑的开放式面试问题
response = client.messages.create(
    model="claude-opus-4-1",
    max_tokens=1000,
    temperature=0.5,
    system="您的任务是基于给定上下文生成一系列深思熟虑的开放式面试问题。问题应该旨在从被面试者那里引出深刻和详细的回应，让他们展示自己的知识、经验和批判性思维技能。避免是/否问题或那些有明显答案的问题。相反，专注于鼓励反思、自我评估和分享具体例子或轶事的问题。",
    messages=formatted_messages,
)

# 打印生成的问题

# 将每个TextBlock的文本连接成单个字符串
content = "".join(block.text for block in response.content)

# 按'\n\n'分割内容
parts = content.split("\n\n")

# 每个部分额外添加一个换行符打印
for part in parts:
    print(part)
    print("\n")

如果此单元格成功，您应该看到基于您原始音频文件的面试问题列表。现在您可以使用Deepgram转录音频并使用Anthropic获取一套面试问题。